## Task 1 - Third-Order Letter Approximation Model

The below code takes in all of the txt files in texts/, filtering out special & numerical characters, then counting how often each sequence of three characters appears and compiles it into a dictionary.

<hr>

In [78]:
import os
import re
import random

texts = []
sanitizedTexts = []
trigrams = {}

for title in os.scandir("texts"): # [2]
    if((title.name.find(".txt")) != -1): # Only attempt to open + add the text file if it's a .txt
        fileContent = open(title) # [1]
        texts.append(fileContent.read()) # Add it to the texts list.
        fileContent.close()

for t in texts:
    sanitizedTexts.append(sanitizeText(t))

trigrams = produceTrigrams(sanitizedTexts)
print(trigrams)

        

{'FRA': 161, 'RAN': 774, 'ANK': 238, 'NKE': 76, 'KEN': 235, 'ENS': 394, 'NST': 443, 'STE': 959, 'TEI': 32, 'EIN': 320, 'INO': 50, 'NOR': 228, 'OR ': 2348, 'R T': 1380, ' TH': 18623, 'THE': 16497, 'HE ': 14242, 'E M': 1946, ' MO': 1592, 'MOD': 68, 'ODE': 113, 'DER': 909, 'ERN': 304, 'RN ': 269, 'N P': 264, ' PR': 1092, 'PRO': 720, 'ROM': 1126, 'OME': 1398, 'MET': 356, 'ETH': 554, 'HEU': 14, 'EUS': 10, 'USB': 12, 'SBY': 4, 'BY ': 952, 'Y M': 473, ' MA': 1719, 'MAR': 313, 'ARY': 164, 'RY ': 1207, 'Y W': 734, ' WO': 1128, 'WOL': 4, 'OLL': 338, 'LLS': 134, 'LST': 49, 'STO': 634, 'TON': 299, 'ONE': 1188, 'NEC': 108, 'ECR': 102, 'CRA': 97, 'RAF': 13, 'AFT': 325, 'FT ': 153, 'T G': 193, ' GO': 614, 'GOD': 94, 'ODW': 7, 'DWI': 32, 'WIN': 440, 'IN ': 3850, 'N S': 776, ' SH': 1952, 'SHE': 1260, 'HEL': 322, 'ELL': 944, 'LLE': 528, 'LEY': 37, 'EY ': 719, 'Y C': 510, ' CO': 2843, 'CON': 1248, 'ONT': 635, 'NTE': 901, 'TEN': 811, 'ENT': 2776, 'NTS': 367, 'TS ': 1232, 'S L': 345, ' LE': 876, 'LET': 382

In [73]:
def sanitizeText(text):
    preamble = " ***" # Ending of the preamble
    postamble = "*** END OF " # Beginning of the postamble.

    # [3] Strips out the preamble and postamble from a text by
    # creating a string in between the preamble and postamble variables.
    sanitizedText = (t[t.index(preamble)+len(preamble):t.index(postamble)]) 
    sanitizedText = re.sub("[^a-zA-Z\\s.\n]", "", sanitizedText) # [4]

    return sanitizedText.upper().rstrip().strip().replace("\n", "") # Set to uppercase, remove new lines, and remove trailing spaces.

In [72]:
def produceTrigrams(texts):
    trigrams = {}

    for text in texts:
        for counter, c in enumerate(text): #Enumerate over the text, used to keep a counter of what character we're on.
            # text[counter:counter+3] gets 3 characters - the current one, and the two afterwards. [5]
            # This basically gives us the "current" trigram.
            currentText = text[counter:counter+3]

            if(trigrams.get(currentText) != None): # If the trigram key already exists, increment it's value.
                trigrams[currentText] = trigrams[currentText] + 1
            else: # If the trigram hasn't been added to the dict yet, add it with the value of 0.
                trigrams[currentText] = 1

    return trigrams
                

## Task 2 - Third-Order Letter Approximation Generation

<hr>

## Task 3 - Analyze The Model

<hr>

## Task 4 - Export Model

<hr>

## References

- [1] - Reading text files in Python: https://www.w3schools.com/python/python_file_open.
- [2] - Using Python's OS Module: https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
- [3] - Finding a string between two substrings (used for removing Pre/Postamble): https://stackoverflow.com/a/51456576
- [4] - Using regex in Python: https://www.w3schools.com/python/python_regex.asp
- [5] - Slicing text in Python: https://stackoverflow.com/a/509295
- 
- 

<hr>